# Face Landmarks

Using a subset of [UMD Faces](http://umdfaces.io) database to train a face landmarks predicting model.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from functools import partial
from multiprocessing import Pool
from os.path import join
from pathlib import Path
from copy import deepcopy
import random
import math
import sys

In [ ]:
try:
    old_path
except NameError:
    old_path = sys.path
    new_path = [str(Path.cwd()/'loop')] + old_path
    sys.path = new_path

In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv
import PIL.Image
from imageio import imread
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from tqdm import tqdm_notebook as tqdm

import torch

In [ ]:
from loop import train
from loop.config import defaults
from basedir import DATA, META, CROPPED, NUM_LANDMARKS

In [ ]:
SEED = 1
DEVICE = torch.device('cuda:1')
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
defaults.device = DEVICE

---
## Dataset First Glance

In [ ]:
meta = pd.read_csv(META)
meta.columns = meta.columns.str.lower()
cols = meta.columns
file_cols = ['subject_id', 'file']
face_cols = cols[cols.str.startswith('face')].tolist()
x_cols = cols[cols.str.startswith('p') & cols.str.endswith('x')].tolist()
y_cols = cols[cols.str.startswith('p') & cols.str.endswith('y')].tolist()
faces_df = meta[file_cols + face_cols + x_cols + y_cols]
faces_df.head()

In [ ]:
def show(i, ax=None):
    global faces_df, DATA
    r = faces_df.loc[i]
    img = imread(DATA/r.file)
    x_pts = [r[k] for k in r.keys() if k[0] == 'p' and k[-1] == 'x']
    y_pts = [r[k] for k in r.keys() if k[0] == 'p' and k[-1] == 'y']
    if ax is None:
        f, ax = plt.subplots(1, 1, figsize=(8, 8))
    rc = Rectangle(xy=(r.face_x, r.face_y), 
                   width=r.face_width, height=r.face_height,
                   edgecolor='red', fill=False, lw=5)
    ax.imshow(img)
    ax.scatter(x_pts, y_pts, edgecolor='white', color='lightgreen', alpha=0.8)    
    ax.add_patch(rc)

In [ ]:
show(3)

In [ ]:
def show_grid(n=3, figsize=(10, 10)):
    global faces_df
    f, axes = plt.subplots(n, n, figsize=figsize)
    indicies = np.random.choice(len(faces_df), n ** 2, replace=False)
    for i, ax in zip(indicies, axes.flat):
        show(i, ax=ax)
        ax.set_axis_off()

In [ ]:
show_grid()

Ok, it seems that each sample includes (at least) one human in various poses with various backgrounds. Therefore, our first goal is to convert this dataset into a more suitable format before we processed with training the model. The most straightforward way to do so is to _crop_ the faces only and save them into smaller files.

---

In [ ]:
def split(target):
    return target[:NUM_LANDMARKS//2], target[NUM_LANDMARKS//2:]

In [ ]:
def create_dataset_item(info, root):
    """Creates a dictionary with face and landmarks coordinates from data frame record."""
    
    return {
        'subject_id': info.subject_id,
        'image_path': str(root/info.file),
        'x_pos': [info[k] for k in info.keys() if k[0] == 'p' and k[-1] == 'x'],
        'y_pos': [info[k] for k in info.keys() if k[0] == 'p' and k[-1] == 'y'],
        'face': (info.face_x, info.face_y, info.face_width, info.face_height)}

In [ ]:
class FaceLandmarks:
    
    def __init__(self, info, root, transforms=None):
        self.transforms = transforms
        self.items = [create_dataset_item(record, DATA) for _, record in meta.iterrows()]
    
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, item):
        record = self.items[item]
        img = imread(record['image_path'])
        pts = np.array(record['x_pos'] + record['y_pos'], dtype='float32')
        if self.transforms:
            img, pts = self.transforms(img, pts)
        return img, pts

In [ ]:
ds = FaceLandmarks(meta, DATA)

In [ ]:
test_img, test_pts = ds[3]

In [ ]:
plt.imshow(test_img)
plt.scatter(test_pts[:NUM_LANDMARKS//2], test_pts[NUM_LANDMARKS//2:])

In [ ]:
xs, ys = split(test_pts)
l, r, t, b = xs.min(), xs.max(), ys.min(), ys.max()
new_w, new_h = 224, 224
src = np.float32([(l, t), (r, t), (r, b), (l, b)])
dst = np.float32([[0, 0], [new_w-1, 0], [new_w-1, new_h-1], [0, new_h-1]])
m = cv.getPerspectiveTransform(src, dst)

In [ ]:
plt.imshow(cv.warpPerspective(test_img, m, (new_w, new_h)), interpolation='bicubic')

In [ ]:
def binomial(p):
    return np.random.rand() > p

In [ ]:
def rotation_matrix(x, y=None, angle=5):
    h, w = x.shape[:2]
    minmax = (-angle, angle) if isinstance(angle, int) else angle
    a = np.random.uniform(*minmax)
    m = cv.getRotationMatrix2D((w/2, h/2), a, 1)
    m = np.r_[m, [[0, 0, 1]]]
    return m

In [ ]:
def shift_matrix(x, y=None, shift=0.01):
    h, w = x.shape[:2]
    sx, sy = (shift, shift) if isinstance(shift, float) else shift
    shift_x = np.random.randint(-w*sx, w*sx)
    shift_y = np.random.randint(-h*sy, h*sy)
    m = np.float32([
        [1, 0, shift_x],
        [0, 1, shift_y],
        [0, 0, 1]]) 
    return m

In [ ]:
def mirror_matrix(x, y=None, horizontal=True):
    h, w = x.shape[:2]
    c1, c2 = (-1, 1) if horizontal else (1, -1)
    s1, s2 = (w, 0) if horizontal else (0, h)
    return np.float32([[c1, 0, s1], [0, c2, s2], [0, 0, 1]])

In [ ]:
def perspective_matrix(x, y=None, percentage=(0.05, 0.12)):
    h, w = x.shape[:2]
    
    def rx(): 
        return int(w*np.random.uniform(*percentage))
    
    def ry(): 
        return int(h*np.random.uniform(*percentage))
    
    tl = [0   + rx(), 0   + ry()]
    tr = [w-1 - rx(), 0   + ry()]
    br = [w-1 - rx(), h-1 - ry()]
    bl = [0   + rx(), h-1 - ry()]
    src = np.float32([tl, tr, br, bl])
    dst = np.float32([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]])
    m = cv.getPerspectiveTransform(src, dst)
    return m

In [ ]:
def crop_face(x, y, size=224):
    pts_x, pts_y = split(y)
    l, r, t, b = pts_x.min(), pts_x.max(), pts_y.min(), pts.y_max()
    new_w, new_h = (size, size) if isinstance(size, int) else size
    src = np.float32([(l, t), (r, t), (r, b), (l, b)])
    dst = np.float32([[0, 0], [new_w-1, 0], [new_w-1, new_h-1], [0, new_h-1]])
    m = cv.getPerspectiveTransform(src, dst)
    return m

In [ ]:
aug_conf = [
    {'name': 'rotation', 'angle': 10},
    {'name': 'mirror'},
    {'name': 'perspective', 'percentage': (0.05, 0.15)},
    {'name': 'crop_face'}
]

In [ ]:
class MatrixAugmentation:
    
    def __init__(self, conf, default_prob=0.5):
        augmentations = {
            'rotation': rotation_matrix,
            'shift': shift_matrix,
            'mirror': mirror_matrix,
            'perspective': perspective_matrix}
        
        pipe = []
        for params in deepcopy(conf):
            name = params.pop('name')
            func = augmentations.get(name)
            if func is None:
                raise ValueError(f'unknown augmentation function: {name}')
            p = params.pop('p', default_prob)
            pipe.append((p, func, params))

        self.pipe = pipe
        
    def __call__(self, image, target=None):
        h, w = image.shape[:2]
        
        m = np.eye(3)
        for p, func, params, transform in self.pipe:
            if binomial(p):
                tr_x, tr_y = transform
                m = func(w, h, **params) @ m
        
        aug_image = cv.warpPerspective(image, m_image, (w, h))
        if target is not None:
            new = target.copy()
            n = len(target)//2
            for i in range(n):
                x, y = target[i], target[i + n]
                denom = m[2][0]*x + m[2][1]*y + m[2][2]
                new[i] = (m[0][0]*x + m[0][1]*y + m[0][2])/denom
                new[i + n] = (m[1][0]*x + m[1][1]*y + m[1][2])/denom
            target = new
            
        return aug_image, target

In [ ]:
# class CropToFace:
#     def __init__(self, pad=0):
#         self.pad = pad
    
#     def __call__(self, image, target):
#         xs, ys = split(target)
#         x_min, x_max, y_min, y_max = xs.min(), xs.max(), ys.min(), ys.max()
#         if self.pad:
#             x_min -= self.pad
#             x_max -= self.pad
#             y_min += self.pad
#             y_max += self.pad 
#         src = np.float32([
#             [x_min, y_min], 
#             [x_max, y_min],
#             [x_max, y_max],
#             [x_min, y_max]
#         ])
#         dst = np.float32([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]])
#         m = cv.getPerspectiveTransform(src, dst)
        

In [ ]:
pipe = MatrixAugmentation(aug_conf)

In [ ]:
aug_img, aug_pts = pipe(test_img, test_pts)
plt.imshow(aug_img)
plt.scatter(aug_pts[:NUM_LANDMARKS//2], aug_pts[NUM_LANDMARKS//2:])

---

## Images Cropping

In [ ]:
def create_meta(unique_id, info, root, output):
    """Creates a dictionary with face and landmarks coordinates from data frame record.
    
    These records with all required meta-information are passed into images generating function.
    
    Args:
        info: Object or namedtuple with information about image.
        root: An absolute path to the folder with annotated images.
        output: Path to the folder where to save processed images.
    
    """
    i = unique_id
    return {
        'index': i,
        'subject_id': info.subject_id,
        'input_path': root/info.file,
        'output_image': output/f'{i}.jpeg',
        'output_points': output/f'{i}.txt',
        'x_pos': [info[k] for k in info.keys() if k[0] == 'p' and k[-1] == 'x'],
        'y_pos': [info[k] for k in info.keys() if k[0] == 'p' and k[-1] == 'y'],
        'face': (info.face_x, info.face_y, info.face_width, info.face_height)
    }

In [ ]:
def as_is(xs, ys, w, h):
    return xs, ys

In [ ]:
def to_centered(xs, ys, w, h):
    return 2*xs/w - 1, 2*ys/h - 1

In [ ]:
def to_absolute(xs, ys, w, h):
    return w*(xs + 1)/2., h*(ys + 1)/2. 

In [ ]:
def crop_image(record, centered=True):
    converter = to_centered if centered else as_is
    x, y, w, h = record['face']
    img = PIL.Image.open(record['input_path'])
    box = x, y, x+w, y+h
    cropped = img.crop(box=box)
    x_pos, y_pos = [np.array(record[k]) for k in ('x_pos', 'y_pos')]
    x_pos, y_pos = converter(x_pos - x, y_pos - y, w, h)
    record.update({'cropped': cropped, 'xy': (x_pos, y_pos), 'centered': centered})
    return record

In [ ]:
def show(record, ax=None):
    if ax is None:
        f, ax = plt.subplots(1, 1)
    conv = to_absolute if record['centered'] else as_is
    img = record['cropped']
    h, w = img.size
    ax.imshow(img)
    ax.scatter(*conv(*record['xy'], w, h), color='lightgreen', edgecolor='white', alpha=0.8)
    ax.set_title(record['index'])
    ax.set_axis_off()

In [ ]:
def save_image(record, fmt='%.4f', delimieter=','):
    cropped, (x_pos, y_pos) = record['cropped'], record['xy']
    cropped.save(record['output_image'], format='jpeg')
    np.savetxt(record['output_points'], np.c_[x_pos, y_pos], fmt='%.4f')

In [ ]:
record = create_meta_record(3, meta.loc[3], DATA, CROPPED)
record = crop_image(record)
show(record)

In [ ]:
def read_ordered_files(path, pattern):
    return list(sorted([fn for fn in path.glob(pattern)], key=lambda filename: int(filename.stem)))    

In [ ]:
def create_face_landmarks_dataset(
    input_dir: Path,
    output_dir: Path,
    faces_info: pd.DataFrame
):
    if output_dir.exists():
        print(f'The data is already prepared. Reading from folder: {output_dir}')
        images = [str(x) for x in read_ordered_files(output_dir, '*.jpeg')]
        points = [str(x) for x in read_ordered_files(output_dir, '*.txt')]
        if not images or not points:
            print('Warning: no files in output folder! Delete the folder and try again')
        data = pd.DataFrame({'image': images, 'points': points})
    
    else:
        output_dir.mkdir(parents=True)
        cols = faces_info.columns
        n = len(faces_info)
        print('Running parallel execution pool...')
        worker = partial(create_cropped_image, input_dir=input_dir, output_dir=output_dir)
        with Pool() as pool:
            results = list(pool.imap(worker, tqdm(faces_info.iterrows(), total=n)))
        data = pd.DataFrame(results)
        print(f'Done! Output folder: {output_dir}')
    
    data['order'] = data.image.map(lambda s: int(Path(s).stem))
    data.sort_values(by='order', inplace=True)
    return data

        
def create_cropped_image(item, input_dir, output_dir):
    i, x = item
    record = create_meta(i, x, input_dir, output_dir)
    record = crop_image(record)
    save_image(record)
    return {'image': str(record['output_image']), 'points': str(record['output_points'])}

In [ ]:
landmarks_df = create_face_landmarks_dataset(DATA, CROPPED, meta)